### Application Programming Interfaces
- is a set of commands, functions, protocols and objects that programmers can use to create software or interact with an external system
- [Your program] Request <-(API)-> Response [External System]

#### API Endpoint: From where to obtain the data
#### API Request: Trying to withdraw some data
#### 


#### Response Codes
- 1XX : Hold on
- 2XX: Here You Go (OK)
- 3XX: Go Away (No Permission)
- 4XX: You screwed up (User)
- 5XX: I Screwed Up (Server)

In [8]:
import requests

# This will return a response code
response = requests.get(url='http://api.open-notify.org/iss-now.json') # API Request
response.raise_for_status()

longitude = response.json()["iss_position"]["longitude"] # Data retrieved
print(longitude)


-160.2860


#### Use this website to find out Current position: https://www.gps-coordinates.net/

#### API Parameters: To pass in specific queries to get specific API Response
- Endpoint?ParamName=Value&...
#### Sunrise and Sunset Time

In [20]:
from datetime import datetime
import requests

response = requests.get(url="http://api.open-notify.org/iss-now.json")
response.raise_for_status()
data = response.json()

iss_latitude = float(data["iss_position"]["latitude"])
iss_longitude = float(data["iss_position"]["longitude"])

MY_LAT = 28.6138954
MY_LONG = 77.2090057

parameters = {
    'lat': MY_LAT,
    'lng': MY_LONG,
    'formatted': 0
}

response = requests.get("https://api.sunrise-sunset.org/json", params=parameters)
response.raise_for_status()
data = response.json()

sunrise = int(data['results']['sunrise'].split("T")[1].split(":")[0]) # only getting the sunset hour 
sunset = int(data['results']['sunset'].split("T")[1].split(":")[0]) # only getting the sunset hour

time_now = datetime.now()


00 12 20


#### ISS Overhead Notifier Project

In [ ]:
import smtplib
import time
import requests
from datetime import datetime

MY_LAT = 28.6138954 # Your latitude
MY_LONG = 77.2090057 # Your longitude

def is_iss_overhead():
    response = requests.get(url="http://api.open-notify.org/iss-now.json")
    response.raise_for_status()
    data = response.json()

    iss_latitude = float(data["iss_position"]["latitude"])
    iss_longitude = float(data["iss_position"]["longitude"])

    #Your position is within +5 or -5 degrees of the ISS position.
    if MY_LAT-5 <= iss_latitude <= MY_LAT+5 and MY_LONG-5 <= iss_longitude <= MY_LONG+5:
        return True

def is_night():
    parameters = {
        "lat": MY_LAT,
        "lng": MY_LONG,
        "formatted": 0,
    }

    response = requests.get("https://api.sunrise-sunset.org/json", params=parameters)
    response.raise_for_status()
    data = response.json()
    sunrise = int(data["results"]["sunrise"].split("T")[1].split(":")[0])
    sunset = int(data["results"]["sunset"].split("T")[1].split(":")[0])

    time_now = datetime.now().hour()

    if time_now >= sunset or time_now <= sunrise:
        return True

while True:
    time.sleep(60)
    if is_iss_overhead and is_night():
        connection = smtplib.SMTP("smtp.gmail.com")
        connection.starttls()
        connection.login('myemail@gmail.com', 'password') # This is not working in my case anyways
        connection.sendmail(
            from_addr='myemail@gmail.com',
            to_addrs='myemail@gmail.com'
            msg="Subject:Look Up👆\n\nThe ISS is above you in the sky."
        )